In [1]:
import requests
import os
import numpy as np
import pandas as pd

%matplotlib inline

The NOAA publishes historical weather data. Details can be found in the [readme](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt). In this tutorial, we explore how to quickly download data in a format similar to what can be ordered through their [search tool](https://www.ncei.noaa.gov/cdo-web/) (which can sometimes take days to send a download link).

In [2]:
response=requests.get('https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt') #stations in the network

In [3]:
response.status_code #it worked!

200

In [4]:
response.url

'https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt'

In [5]:
with open('stationdata.txt', 'w') as file: #write stations to file for future reference
    file.write(response.text) #or use response.content in mode 'wb'

In [6]:
#from readme
#IV. FORMAT OF "ghcnd-stations.txt"

#------------------------------
#Variable   Columns   Type
#------------------------------
#ID            1-11   Character
#LATITUDE     13-20   Real
#LONGITUDE    22-30   Real
#ELEVATION    32-37   Real
#STATE        39-40   Character
#NAME         42-71   Character
#GSN FLAG     73-75   Character
#HCN/CRN FLAG 77-79   Character
#WMO ID       81-85   Character
#------------------------------

In [7]:
#cannot use pd.read_csv because the file is not in the correct format
#use np.genfromtxt instead to read from fixed-width text file
#fields are unicode strings of prescribed length or double

stations = np.genfromtxt(os.getcwd()+'/stationdata.txt', delimiter=[11,9,10,7,3,31,4,4,6],
                                         names=['id','latitude','longitude','elevation','state','name',
                                                'gsn','hcn','wmo'],
                                         dtype=['U11','d','d','d','U3','U31','U4','U4','U6'],
                                         autostrip=True)

We can now pick stations for which to download historical weather data. In this example, we download data from the nearest station for Watertown, MA.

In [8]:
stations[stations['name']=='WATERTOWN'] #this only shows Watertown, MN and Watertown, NY

array([('USC00218713', 44.9667, -93.85  , -999.9, 'MN', 'WATERTOWN', '', '', ''),
       ('USC00309000', 43.9761, -75.8753,  151.5, 'NY', 'WATERTOWN', '', 'HCN', '')],
      dtype=[('id', '<U11'), ('latitude', '<f8'), ('longitude', '<f8'), ('elevation', '<f8'), ('state', '<U3'), ('name', '<U31'), ('gsn', '<U4'), ('hcn', '<U4'), ('wmo', '<U6')])

In [9]:
stations[np.char.find(stations['name'],'WATERTOWN')==0] #any other stations beginning with 'WATERTOWN'?

array([('US1CTLT0010', 41.5995, -73.1378,  245.7, 'CT', 'WATERTOWN 1.1 WSW', '', '', ''),
       ('US1CTLT0014', 41.5991, -73.1166,  178.9, 'CT', 'WATERTOWN 0.5 S', '', '', ''),
       ('US1MAMD0119', 42.3711, -71.1995,   16.5, 'MA', 'WATERTOWN 1.1 W', '', '', ''),
       ('US1MNCV0008', 44.9663, -93.8489,  295.7, 'MN', 'WATERTOWN 0.5 NNW', '', '', ''),
       ('US1NYJF0010', 43.9819, -75.934 ,  122.2, 'NY', 'WATERTOWN 1.3 WNW', '', '', ''),
       ('US1NYJF0020', 43.9708, -75.909 ,  151.8, 'NY', 'WATERTOWN 0.2 SSE', '', '', ''),
       ('US1NYJF0032', 43.9669, -75.892 ,  159.1, 'NY', 'WATERTOWN 1.0 ESE', '', '', ''),
       ('US1SDCD0001', 44.959 , -97.0247,  580. , 'SD', 'WATERTOWN 7.6 ENE', '', '', ''),
       ('US1SDCD0002', 44.9429, -97.1461,  526.1, 'SD', 'WATERTOWN 2.3 NNE', '', '', ''),
       ('US1SDCD0005', 45.0043, -97.0857,  542.8, 'SD', 'WATERTOWN 7.5 NNE', '', '', ''),
       ('US1SDCD0008', 44.9074, -97.1116,  534.3, 'SD', 'WATERTOWN 2.7 E', '', '', ''),
       ('US1SDCD

There is one station in Watertown, MA, but it is not part of the Historical Climatology Network (HCN). It may only have limited data available. Proceed to request the daily weather data records through the API and write to file.

In [10]:
response=requests.get('https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/all/US1MAMD0119.dly')
with open('WATERTOWN.dly', 'w') as file:
    file.write(response.text) #or use response.content in mode 'wb'

In [11]:
#format from readme

#------------------------------
#Variable   Columns   Type
#------------------------------
#ID            1-11   Character
#YEAR         12-15   Integer
#MONTH        16-17   Integer
#ELEMENT      18-21   Character
#VALUE1       22-26   Integer
#MFLAG1       27-27   Character
#QFLAG1       28-28   Character
#SFLAG1       29-29   Character
#VALUE2       30-34   Integer
#MFLAG2       35-35   Character
#QFLAG2       36-36   Character
#SFLAG2       37-37   Character
#  .           .          .
#  .           .          .
#  .           .          .
#VALUE31    262-266   Integer
#MFLAG31    267-267   Character
#QFLAG31    268-268   Character
#SFLAG31    269-269   Character
#------------------------------

#These variables have the following definitions:
#
#ID         is the station identification code.  Please see "ghcnd-stations.txt"
#           for a complete list of stations and their metadata.
#YEAR       is the year of the record.
#
#MONTH      is the month of the record.
#
#ELEMENT    is the element type.   There are five core elements as well as a number
#           of addition elements.  
#   
#   The five core elements are:
#
#   PRCP = Precipitation (tenths of mm)
#   SNOW = Snowfall (mm)
#   SNWD = Snow depth (mm)
#   TMAX = Maximum temperature (tenths of degrees C)
#   TMIN = Minimum temperature (tenths of degrees C)


#   WESD = Water equivalent of snow on the ground (tenths of mm)
#   WESF = Water equivalent of snowfall (tenths of mm)

#... and so on through the 31st day of the month.  Note: If the month has less 
#than 31 days, then the remaining variables are set to missing (e.g., for April, 
#VALUE31 = -9999, MFLAG31 = blank, QFLAG31 = blank, SFLAG31 = blank).


In [12]:
def dly_to_df(filename):
    # load the fixed-width file following the format in readme.txt
    # and label the columns
    # note that each row contains an entire month (31 days, some data will be missing)
    colnames=['STATION','YEAR','MONTH','ELEMENT']
    for i in range(1,32): #how to do this in a single list comprehension?
        colnames.extend([f'day{i}', f'm{i}', f'q{i}', f's{i}'])
    data = np.genfromtxt(filename,
                      delimiter=[11,4,2,4] + [5,1,1,1]*31,
                      names=colnames,
                      dtype=['U11','i','i','U4'] + ['d','U1','U1','U1']*31,
                      autostrip=True)
    pdata=pd.DataFrame(data) #convert to pandas DataFrame
    return pdata

In [13]:
pdata=dly_to_df('WATERTOWN.dly') #this station only provides precipitation data, no temperature data
pdata

We now reformat this data frame step by step, so that each day of data shows up in a separate row, with each element a column, and each set of element attributes another column.

In [15]:
pdata = pd.melt(pdata, id_vars=['STATION','YEAR','MONTH','ELEMENT']) #first convert to long format
pdata.head()

,STATION,YEAR,MONTH,ELEMENT,variable,value
0,US1MAMD0119,2018,5,PRCP,day1,-9999
1,US1MAMD0119,2018,5,SNOW,day1,-9999
2,US1MAMD0119,2018,5,SNWD,day1,-9999
3,US1MAMD0119,2018,5,WESD,day1,-9999
4,US1MAMD0119,2018,5,WESF,day1,-9999


In [16]:
pdata['DAY']=pdata['variable'].apply(lambda x: int(x[3:]) if len(x)>3 else int(x[1:])) #create a separate variable 'DAY'

In [17]:
pdata['variable']=pdata['variable'].apply(lambda x: x[:3] if len(x)>3 else x[0]) #and reduce the 'variable' column to 'day'
#or flag identifiers 'm', 'q', 's'

In [18]:
pdata.head()

,STATION,YEAR,MONTH,ELEMENT,variable,value,DAY
0,US1MAMD0119,2018,5,PRCP,day,-9999,1
1,US1MAMD0119,2018,5,SNOW,day,-9999,1
2,US1MAMD0119,2018,5,SNWD,day,-9999,1
3,US1MAMD0119,2018,5,WESD,day,-9999,1
4,US1MAMD0119,2018,5,WESF,day,-9999,1


In [19]:
#every date for every element has 'variable' entries 'day', 'm','q','s'
#use 'STATION','YEAR','MONTH','DAY','ELEMENT' as row index and the entries in 'variable' to create new columns with values
#associated from the 'value' column
pdata=pd.pivot(pdata,index=['STATION','YEAR','MONTH','DAY','ELEMENT'],columns=['variable'], values='value')
pdata.head()

variable                             day m q s
STATION     YEAR MONTH DAY ELEMENT            
US1MAMD0119 2018 5     1   PRCP    -9999      
                           SNOW    -9999      
                           SNWD    -9999      
                           WESD    -9999      
                           WESF    -9999

In [20]:
pdata.rename(columns={'day':'value'}, inplace=True)
pdata.columns

Index(['value', 'm', 'q', 's'], dtype='object', name='variable')

In [21]:
pdata.reset_index(inplace=True) #simple index for each row, with 'STATION', 'YEAR','MONTH','DAY','ELEMENT' columns
pdata.head()

variable,STATION,YEAR,MONTH,DAY,ELEMENT,value,m,q,s
0,US1MAMD0119,2018,5,1,PRCP,-9999,,,
1,US1MAMD0119,2018,5,1,SNOW,-9999,,,
2,US1MAMD0119,2018,5,1,SNWD,-9999,,,
3,US1MAMD0119,2018,5,1,WESD,-9999,,,
4,US1MAMD0119,2018,5,1,WESF,-9999,,,


In [22]:
#throw out data for all days with 'invalid' values
pdata = pdata[pdata.value != -9999]
#such days are, e.g., Feb 30

In [23]:
# make a column 'DATE' out of year, month, day
pdata['DATE'] = pd.to_datetime(pdata[['YEAR','MONTH','DAY']])
pdata.head()

variable,STATION,YEAR,MONTH,DAY,ELEMENT,value,m,q,s,DATE
10,US1MAMD0119,2018,5,3,PRCP,0,,,N,2018-05-03
11,US1MAMD0119,2018,5,3,SNOW,0,,,N,2018-05-03
12,US1MAMD0119,2018,5,3,SNWD,0,,,N,2018-05-03
13,US1MAMD0119,2018,5,3,WESD,0,,,N,2018-05-03
14,US1MAMD0119,2018,5,3,WESF,0,,,N,2018-05-03


In [24]:
pdata.columns.name=None #the identifier 'variable' for the column names is not needed
pdata.head()

,STATION,YEAR,MONTH,DAY,ELEMENT,value,m,q,s,DATE
10,US1MAMD0119,2018,5,3,PRCP,0,,,N,2018-05-03
11,US1MAMD0119,2018,5,3,SNOW,0,,,N,2018-05-03
12,US1MAMD0119,2018,5,3,SNWD,0,,,N,2018-05-03
13,US1MAMD0119,2018,5,3,WESD,0,,,N,2018-05-03
14,US1MAMD0119,2018,5,3,WESF,0,,,N,2018-05-03


In [25]:
#forget separate YEAR,MONTH,DAY columns
pdata=pdata[['STATION','DATE','ELEMENT','value','m','q','s']]
pdata.head()

,STATION,DATE,ELEMENT,value,m,q,s
10,US1MAMD0119,2018-05-03,PRCP,0,,,N
11,US1MAMD0119,2018-05-03,SNOW,0,,,N
12,US1MAMD0119,2018-05-03,SNWD,0,,,N
13,US1MAMD0119,2018-05-03,WESD,0,,,N
14,US1MAMD0119,2018-05-03,WESF,0,,,N


In [26]:
#consolidate the flags for each element in an 'ATTRIBUTES' column
pdata['ATTRIBUTES']=pdata['m']+','+pdata['q']+','+pdata['s']
pdata.head()

,STATION,DATE,ELEMENT,value,m,q,s,ATTRIBUTES
10,US1MAMD0119,2018-05-03,PRCP,0,,,N,",,N"
11,US1MAMD0119,2018-05-03,SNOW,0,,,N,",,N"
12,US1MAMD0119,2018-05-03,SNWD,0,,,N,",,N"
13,US1MAMD0119,2018-05-03,WESD,0,,,N,",,N"
14,US1MAMD0119,2018-05-03,WESF,0,,,N,",,N"


In [27]:
#forget separate m, q, s columns
pdata=pdata[['STATION','DATE','ELEMENT','value','ATTRIBUTES']]
pdata.head()

,STATION,DATE,ELEMENT,value,ATTRIBUTES
10,US1MAMD0119,2018-05-03,PRCP,0,",,N"
11,US1MAMD0119,2018-05-03,SNOW,0,",,N"
12,US1MAMD0119,2018-05-03,SNWD,0,",,N"
13,US1MAMD0119,2018-05-03,WESD,0,",,N"
14,US1MAMD0119,2018-05-03,WESF,0,",,N"


In [28]:
#now, for each day, get all elements and attributes to show up in a row
pdata = pdata.pivot(index=['STATION','DATE'], columns='ELEMENT')
pdata.head()

value                     ATTRIBUTES                 \
ELEMENT                 PRCP SNOW SNWD WESD WESF       PRCP SNOW SNWD WESD   
STATION     DATE                                                             
US1MAMD0119 2018-05-03     0    0    0    0    0        ,,N  ,,N  ,,N  ,,N   
            2018-05-04    13    0    0    0    0        ,,N  ,,N  ,,N  ,,N   
            2018-05-05     3    0    0    0    0        ,,N  ,,N  ,,N  ,,N   
            2018-05-06     0    0    0    0    0       T,,N  ,,N  ,,N  ,,N   
            2018-05-07    71    0    0    0    0        ,,N  ,,N  ,,N  ,,N   

                             
ELEMENT                WESF  
STATION     DATE             
US1MAMD0119 2018-05-03  ,,N  
            2018-05-04  ,,N  
            2018-05-05  ,,N  
            2018-05-06  ,,N  
            2018-05-07  ,,N

In [29]:
pdata.columns

MultiIndex([(     'value', 'PRCP'),
            (     'value', 'SNOW'),
            (     'value', 'SNWD'),
            (     'value', 'WESD'),
            (     'value', 'WESF'),
            ('ATTRIBUTES', 'PRCP'),
            ('ATTRIBUTES', 'SNOW'),
            ('ATTRIBUTES', 'SNWD'),
            ('ATTRIBUTES', 'WESD'),
            ('ATTRIBUTES', 'WESF')],
           names=[None, 'ELEMENT'])

In [30]:
#rename the columns
pdata.rename(columns={'value':''}, inplace=True)
pdata.columns

MultiIndex([(          '', 'PRCP'),
            (          '', 'SNOW'),
            (          '', 'SNWD'),
            (          '', 'WESD'),
            (          '', 'WESF'),
            ('ATTRIBUTES', 'PRCP'),
            ('ATTRIBUTES', 'SNOW'),
            ('ATTRIBUTES', 'SNWD'),
            ('ATTRIBUTES', 'WESD'),
            ('ATTRIBUTES', 'WESF')],
           names=[None, 'ELEMENT'])

In [31]:
pdata.columns = ['_'.join(col).rstrip('_') for col in [c[::-1] for c in pdata.columns.values]]
pdata.columns

Index(['PRCP', 'SNOW', 'SNWD', 'WESD', 'WESF', 'PRCP_ATTRIBUTES',
       'SNOW_ATTRIBUTES', 'SNWD_ATTRIBUTES', 'WESD_ATTRIBUTES',
       'WESF_ATTRIBUTES'],
      dtype='object')

In [32]:
#alphabetical order of columns
pdata=pdata[list(pdata.columns.sort_values())]
pdata.head()

PRCP PRCP_ATTRIBUTES SNOW SNOW_ATTRIBUTES SNWD  \
STATION     DATE                                                        
US1MAMD0119 2018-05-03    0             ,,N    0             ,,N    0   
            2018-05-04   13             ,,N    0             ,,N    0   
            2018-05-05    3             ,,N    0             ,,N    0   
            2018-05-06    0            T,,N    0             ,,N    0   
            2018-05-07   71             ,,N    0             ,,N    0   

                       SNWD_ATTRIBUTES WESD WESD_ATTRIBUTES WESF  \
STATION     DATE                                                   
US1MAMD0119 2018-05-03             ,,N    0             ,,N    0   
            2018-05-04             ,,N    0             ,,N    0   
            2018-05-05             ,,N    0             ,,N    0   
            2018-05-06             ,,N    0             ,,N    0   
            2018-05-07             ,,N    0             ,,N    0   

                       WESF_ATTRIBUTES  
STATION     DATE                        
US1MAMD0119 2018-05-03             ,,N  
            2018-05-04             ,,N  
            2018-05-05             ,,N  
            2018-05-06             ,,N  
            2018-05-07             ,,N

In [33]:
pdata.reset_index(inplace=True)
pdata.head()

,STATION,DATE,PRCP,PRCP_ATTRIBUTES,SNOW,SNOW_ATTRIBUTES,SNWD,SNWD_ATTRIBUTES,WESD,WESD_ATTRIBUTES,WESF,WESF_ATTRIBUTES
0,US1MAMD0119,2018-05-03,0,",,N",0,",,N",0,",,N",0,",,N",0,",,N"
1,US1MAMD0119,2018-05-04,13,",,N",0,",,N",0,",,N",0,",,N",0,",,N"
2,US1MAMD0119,2018-05-05,3,",,N",0,",,N",0,",,N",0,",,N",0,",,N"
3,US1MAMD0119,2018-05-06,0,"T,,N",0,",,N",0,",,N",0,",,N",0,",,N"
4,US1MAMD0119,2018-05-07,71,",,N",0,",,N",0,",,N",0,",,N",0,",,N"


In [34]:
pdata.to_csv('WATERTOWN.csv',index=False) #save to .csv

In [35]:
#create a function aggregating all these steps
def dly_to_csv(filename, target=None):
    # load the fixed-width file following the format in readme.txt
    # and label the columns
    # note that each row contains an entire month (31 days, some data will be missing)
    colnames=['STATION','YEAR','MONTH','ELEMENT']
    for i in range(1,32): 
        colnames.extend([f'day{i}', f'm{i}', f'q{i}', f's{i}'])
    data = np.genfromtxt(filename,
                      delimiter=[11,4,2,4] + [5,1,1,1]*31,
                      names=colnames,
                      dtype=['U11','i','i','U4'] + ['d','U1','U1','U1']*31,
                      autostrip=True)
    pdata=pd.DataFrame(data) #convert to pandas DataFrame
    
    pdata = pd.melt(pdata, id_vars=['STATION','YEAR','MONTH','ELEMENT'])
    pdata['DAY']=pdata['variable'].apply(lambda x: int(x[3:]) if len(x)>3 else int(x[1:]))
    pdata['variable']=pdata['variable'].apply(lambda x: x[:3] if len(x)>3 else x[0])
    pdata=pd.pivot(pdata,index=['STATION','YEAR','MONTH','DAY','ELEMENT'],columns=['variable'], values='value')
    pdata.rename(columns={'day':'value'}, inplace=True)
    #throw out data for all days with 'invalid' measurements
    pdata = pdata[pdata.value != -9999]
    #such days are, e.g., Feb 30
    pdata.reset_index(inplace=True)
    # make a column 'date' out of year, month, day
    pdata['DATE'] = pd.to_datetime(pdata[['YEAR','MONTH','DAY']])
    pdata.columns.name=None
    #forget separate YEAR,MONTH,DAY columns
    pdata=pdata[['STATION','DATE','ELEMENT','value','m','q','s']]
    pdata['ATTRIBUTES']=pdata['m']+','+pdata['q']+','+pdata['s']
    #forget separate m, q, s columns
    pdata=pdata[['STATION','DATE','ELEMENT','value','ATTRIBUTES']]
    pdata = pdata.pivot(index=['STATION','DATE'], columns='ELEMENT')
    pdata.rename(columns={'value':''}, inplace=True)
    pdata.columns = ['_'.join(col).rstrip('_') for col in [c[::-1] for c in pdata.columns.values]]
    pdata=pdata[list(pdata.columns.sort_values())] #sort column names (elements, attributes) alphabetically
    pdata.reset_index(inplace=True)
    if not target:
        target=filename
    pdata.to_csv(target+'.csv', index=False)

In [36]:
dly_to_csv('WATERTOWN.dly','WATERTOWN')

For comparison, let us try a station that has more data, because it is in the GSN ([GCOS surface network](https://gcos.wmo.int/en/networks/atmospheric/gsn)), HCN (Historical Climatology Network), or WMO (World Meteorological Organization) network of stations.

In [37]:
stations[np.char.find(stations['name'],'BOSTON')==0] #search for stations beginning with 'BOSTON'

array([('ASN00064019', -32.2833,  149.0833, -999.9, '', 'BOSTON (GOLLAN)', '', '', ''),
       ('CA00111090M',  49.8667, -121.4333,  200. , 'BC', 'BOSTON BAR', '', '', ''),
       ('CA001110R04',  49.8667, -121.45  ,  163. , 'BC', 'BOSTON BAR', '', '', ''),
       ('CA1ON000066',  42.9851,  -80.268 ,  236.5, 'ON', 'BOSTON 0.8 SSE', '', '', ''),
       ('US1GATH0005',  30.8403,  -83.8033,   59.7, 'GA', 'BOSTON 3.4 NNW', '', '', ''),
       ('US1MASF0001',  42.357 ,  -71.0671,   13.1, 'MA', 'BOSTON 0.5 WSW', '', '', ''),
       ('US1NYER0065',  42.6547,  -78.7201,  475.8, 'NY', 'BOSTON 1.5 NE', '', '', ''),
       ('US1NYER0166',  42.6548,  -78.703 ,  490.7, 'NY', 'BOSTON 2.5 NE', '', '', ''),
       ('USC00150874',  37.7667,  -85.7   ,  146. , 'KY', 'BOSTON 2 SW', '', '', ''),
       ('USC00150875',  37.7436,  -85.7483,  259.1, 'KY', 'BOSTON 6 SW', '', '', ''),
       ('USC00190768',  42.35  ,  -71.0667,    5.2, 'MA', 'BOSTON', '', '', ''),
       ('USC00440860',  38.5458,  -78.0981,  1

We see there is one station at Boston Logan International Airport in Boston, MA that is in the WMO network.

In [38]:
response=requests.get('https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/all/USW00014739.dly')
with open('BOSTON.dly', 'w') as file:
    file.write(response.text)

In [39]:
dly_to_csv('BOSTON.dly','BOSTON')

In [40]:
df=pd.read_csv('BOSTON.csv',low_memory=False) #low_memory=False suppresses a warning about mixed data types in columns 
df.head()

,STATION,DATE,ACMH,ACMH_ATTRIBUTES,ACSH,ACSH_ATTRIBUTES,AWND,AWND_ATTRIBUTES,FMTM,FMTM_ATTRIBUTES,...,WT17,WT17_ATTRIBUTES,WT18,WT18_ATTRIBUTES,WT19,WT19_ATTRIBUTES,WT21,WT21_ATTRIBUTES,WT22,WT22_ATTRIBUTES
0,USW00014739,1936-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,USW00014739,1936-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,",,X",NaN,NaN,NaN,NaN,NaN,NaN
2,USW00014739,1936-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,USW00014739,1936-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,USW00014739,1936-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,",,X",NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
#look at the column names present and select a list of those starting with P,S,T and second letter R,S,M
cols=list(df.columns.values)
mask=list(pd.Series(cols).str.match('[PST][RNM]'))
cols=[col for i,col in enumerate(cols) if mask[i]]

In [42]:
cols #this picks out the most common elements and their attributes

['PRCP',
 'PRCP_ATTRIBUTES',
 'SNOW',
 'SNOW_ATTRIBUTES',
 'SNWD',
 'SNWD_ATTRIBUTES',
 'TMAX',
 'TMAX_ATTRIBUTES',
 'TMIN',
 'TMIN_ATTRIBUTES']

In [43]:
df=df[['STATION','DATE']+cols]
#select dates from May 2018 where the above data for Watertown starts
df=df[df['DATE']>='2018-05-03']
df.head()

,STATION,DATE,PRCP,PRCP_ATTRIBUTES,SNOW,SNOW_ATTRIBUTES,SNWD,SNWD_ATTRIBUTES,TMAX,TMAX_ATTRIBUTES,TMIN,TMIN_ATTRIBUTES
30073,USW00014739,2018-05-03,23.0,",,W",0.0,",,W",NaN,NaN,322.0,",,W",161.0,",,W"
30074,USW00014739,2018-05-04,0.0,"T,,W",0.0,",,W",NaN,NaN,278.0,",,W",139.0,",,W"
30075,USW00014739,2018-05-05,0.0,",,W",0.0,",,W",NaN,NaN,239.0,",,W",144.0,",,W"
30076,USW00014739,2018-05-06,48.0,",,W",0.0,",,W",NaN,NaN,194.0,",,W",100.0,",,W"
30077,USW00014739,2018-05-07,0.0,",,W",0.0,",,W",NaN,NaN,161.0,",,W",89.0,",,W"


Finally, when working with the data, we may have to reformat some 'ELEMENT' columns. Refer to the [documentation](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt) to find the exact units (such as tenths of degrees Celsius, tenths of mm,...) and convert them to the units of your liking (e.g. degrees Fahrenheit, inches, mm).